In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from torch.utils.data import DataLoader
import random
import pandas as pd
import torch
import numpy as np
import gc
from sklearn.model_selection import train_test_split


print("Total GPU Memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")
print("Available GPU Memory:", torch.cuda.memory_reserved(0) / 1e9, "GB")
print("Allocated GPU Memory:", torch.cuda.memory_allocated(0) / 1e9, "GB")
print("Free (cached) GPU Memory:", torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0) / 1e9, "GB")


/home/aljaz/miniforge3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total GPU Memory: 6.226378752 GB
Available GPU Memory: 0.0 GB
Allocated GPU Memory: 0.0 GB
Free (cached) GPU Memory: 0.0 GB


In [2]:
df = pd.read_csv('../data/raw/Annotation_paragraph-level_utf8.txt', sep='\t')
df = df.dropna()
df

,nid,pid,content,Ann1,Ann2,Ann3,Ann4,Ann5,Ann6,avg_sentiment,sd_sentiment,sentiment
71,7,1,Nemški avtomobilski proizvajalec BMW je kitajs...,1.0,1.0,3.0,2.0,1.0,2.0,1.667,0.816,negative
98,11,1,Ameriška nafta je na azijskem trgu z več kot 8...,1.0,1.0,2.0,1.0,2.0,1.0,1.333,0.516,negative
99,11,2,Cena nafte vztrajno narašča,1.0,1.0,2.0,1.0,1.0,2.0,1.333,0.516,negative
100,11,3,Cena ameriške nafte se je z več kot 81 dolarji...,1.0,1.0,3.0,1.0,1.0,1.0,1.333,0.816,negative
101,11,4,Po mnenju borznih posrednikov je mogoče vzrok ...,2.0,2.0,3.0,3.0,2.0,2.0,2.333,0.516,negative
...,...,...,...,...,...,...,...,...,...,...,...,...
71975,8211,1,Borzno trgovanje bo letos potekalo le še v pet...,2.0,3.0,3.0,2.0,3.0,2.0,2.500,0.548,neutral
71976,8211,2,Na ljubljanski borzi danes ne bo potekalo trgo...,2.0,3.0,3.0,3.0,3.0,3.0,2.833,0.408,neutral
71977,8211,3,Borzno trgovanje bo letos potekalo le še v pet...,2.0,3.0,3.0,2.0,4.0,3.0,2.833,0.753,neutral
71978,8211,4,Dveletne in triletne obveznice z letno obrestn...,2.0,2.0,3.0,3.0,4.0,3.0,2.833,0.753,neutral


In [72]:
testing_paragraphs = df.iloc[:, [2, 11]].to_numpy()



In [3]:

phrases = []

doc_names = ["AutoSentiNews_document-level_01.txt", "AutoSentiNews_document-level_02.txt"]

for doc_name in doc_names:

    documents = pd.read_csv(f'../data/raw/{doc_name}', sep='\t')
    documents = documents.iloc[:, 5]
    documents.columns = ['text']
    documents = documents.str.split('.' ).explode().reset_index(drop=True)
    phrases.extend(documents.to_list())

len(phrases)

1846824

In [5]:
pahrases = np.random.shuffle(phrases)
phrases = phrases[:100000]

In [ ]:
len(phrases)

In [82]:
# remove phrases that have less then 10 words

phrases = [phrase for phrase in phrases if len(phrase.split()) > 10]


In [88]:
df = pd.DataFrame(phrases, columns=['text'])
df.to_csv('../data/processed/phrases.csv', index=False, header=False)

In [89]:
df = pd.read_csv('../data/processed/phrases.csv', header=None)
phrases = df.iloc[:, 0].to_list()
phrases

['Nastaja energetski gigant Družbi Suez in Gaz de France se bosta združili v največjega ponudnika tekočega plina na svetu in zemeljskega v EU',
 ' Vodstvo Sueza se je z GDF dogovorilo o združitvi (Foto: Reuters) Novi koncern bo največji ponudnik tekočega plina na svetu, največji ponudnik zemeljskega plina v EU ter peti po velikosti na evropskem trgu električne energije',
 ' Nadzorna sveta zasebnega francosko-belgijskega koncerna za preskrbo z energijo in vodo Suez ter francoskega državnega plinskega koncerna Gaz de France (GDF) sta v nedeljo zvečer po več kot leto dni trajajočih pogajanjih le odobrila združitev obeh podjetij',
 ' GDF in Suez se bosta združila v enega največjih energetskih koncernov na svetu, sta sporočili upravi obeh družb',
 ' Združitev bodo opravili z zamenjavo delnic v dogovorjenem razmerju 21 delnic GDF za 22 delnic Sueza',
 ' Pričakovani sinergijski učinki znašajo ob enkratnih združitvenih stroških v višini 300 milijonov evrov milijardo evrov po letu 2013',
 ' Nov

In [4]:
torch.cuda.empty_cache() 


In [85]:
torch.cuda.empty_cache() 
# Initialize a pre-trained model
basePath = "/home/aljaz/FAKS/ul-fri-nlp-course-project-sneguljcica/models/"
# modelName = "distilbert-base-uncased"
# modelName = "distiluse-base-multilingual-cased-v2"
# modelName = "all-MiniLM-L12-v2"
modelName = "bert-base-multilingual-uncased"

modelPath = basePath + modelName
# modelPath = "/home/aljaz/FAKS/ul-fri-nlp-course-project-sneguljcica/models/distiluse-base-multilingual-cased-v2"
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# Example data: Replace this with loading your dataset
sentences =phrases
train_data = DenoisingAutoEncoderDataset(sentences)

train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True, drop_last=True)

bert = models.Transformer(modelPath)
pooling = models.Pooling(bert.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[bert, pooling])


print(model)



SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


How to test if trained

In [17]:
X = paragrah[:10000, 0]
y = paragrah[:10000, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_embeddings = model.encode(X_train)
test_embeddings = model.encode(X_test)

#train logistic regression
print("training logistic regression")
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression(max_iter=1000)
lr.fit(train_embeddings, y_train)

y_pred = lr.predict(test_embeddings)
report = classification_report(y_test, y_pred)
print(report)

training logistic regression
              precision    recall  f1-score   support

    negative       0.53      0.27      0.36       466
     neutral       0.71      0.93      0.80      1320
    positive       0.42      0.08      0.14       214

    accuracy                           0.68      2000
   macro avg       0.55      0.43      0.43      2000
weighted avg       0.64      0.68      0.63      2000



In [86]:
print("losses")

train_loss = losses.DenoisingAutoEncoderLoss(model,decoder_name_or_path=modelPath ,tie_encoder_decoder=False)

num_epochs = 1

print('Training the model')
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save(basePath+"tsdae-" + modelName)

losses


Some weights of BertLMHeadModel were not initialized from the model checkpoint at /home/aljaz/FAKS/ul-fri-nlp-course-project-sneguljcica/models/bert-base-multilingual-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'encoder.layer.0.crossattention.output.LayerNorm.bias', 'encoder.layer.0.crossattention.output.LayerNorm.weight', 'encoder.layer.0.crossattention.output.dense.bias', 'encoder.layer.0.crossattention.output.dense.weight', 'encoder.layer.0.crossattention.self.key.bias', 'encoder.layer.0.crossattention.self.key.weight', 'encoder.layer.0.crossattention.self.query.bias', 'encoder.layer.0.crossattention.self.query.weight', 'encoder.layer.0.crossattention.self.value.bias', 'encoder.layer.0.crossattention.self.value.weight', 'encoder.layer.1.crossattention.output.LayerNor

Training the model


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 860.00 MiB. GPU 